# Load packages

In [8]:
# !pip install git+https://github.com/keras-team/keras.git -U
# !pip install seqeval
# !pip install tensorflow-gpu==2.0.0-alpha0
# !pip install tensorflow-hub

# Script

In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import TimeDistributed, Dense, Embedding, Input, Dropout, LSTM, Bidirectional, Lambda
from tensorflow.keras.layers import concatenate
# from sklearn.metrics import recall_score, precision_score
from seqeval.metrics import classification_report as classification_report_entity
from seqeval.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [10]:
data = pd.read_csv("../../data/labeled/iob-labeled-sent-final-060519-v2.csv")
data['split_tag'].value_counts()

train         2214
test           281
validation     252
Name: split_tag, dtype: int64

In [11]:
data.loc[70:80]

,Unnamed: 0,old_para_index,para_index,sent,label,split_tag
70,70,118,56,"""I am sick of Mr Bingley,""",B-OTHER,train
71,71,118,56,cried his wife.,O,train
72,72,119,57,"""I am sorry to hear that; but why did not you ...",B-OTHER,train
73,73,120,58,The astonishment of the ladies was just what h...,O,train
74,74,121,59,"""How good it was in you, my dear Mr Bennet! Bu...",B-START,train
75,75,122,60,"""Now, Kitty, you may cough as much as you choo...",B-OTHER,train
76,76,122,60,"said Mr Bennet; and, as he spoke, he le...",O,train
77,77,123,61,"""What an excellent father you have, girls!""",B-START,train
78,78,123,61,"""I do not know how you will ever make him amen...",I-START,train
79,79,124,62,"""Oh!""",B-OTHER,train


In [12]:
class paragraphGetter(object):
    
    def __init__(self, data):
        self.n_para = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [w for w in s["sent"].values.tolist()]
        self.grouped = self.data.groupby("grouped_para_index").apply(agg_func)
        self.paragraphs = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["grouped_paragraph: {}".format(self.n_para)]
            print(s)
            self.n_para += 1
            return s
        except:
            return None
        
        
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
#             print(p)
            out_i.append(idx2label[p].replace("PAD", "O"))
#           print(out_i)
        out.append(out_i)
    return out

def bertSentenceEmbedding(x):
    embeddings = []
    count = 0
    for para in x:
#     count = 0
#     print(para)
        para_all_embed = np.asarray(bert_embedding(para))[:,1]

        para_embed = []
        for embed in np.asarray(para_all_embed):
            count = 0
            all_embed = [0]*768
            for word_embed in embed:
                count += 1
                all_embed = all_embed + word_embed
                #       print(count)
            sent_embed = all_embed/count
            para_embed.append(sent_embed)

        embeddings.append(para_embed)
    #     print(np.asarray(bert_embedding(para)))
    return embeddings

In [13]:
from tensorflow.keras.models import load_model
import os

loc = '../models/'
model_name = "new_fiction_lstm_bert_v4_0506_1303_epoch30_b4_v4.h5"
model_load = load_model(os.path.join(loc, model_name))

In [14]:
data = pd.read_csv("../../data/labeled/iob-labeled-sent-final-060519-v2.csv")
data = data.fillna(method="ffill")

max_len = 11
num_para_in_group = 4
data = data[data['split_tag']=='test']
data['grouped_para_index'] = data['para_index'].apply(lambda x: int(x/num_para_in_group+1))

getter = paragraphGetter(data)
paragraphs = getter.paragraphs
print(len(paragraphs))

label2idx = {'B-OTHER': 4, 'B-START': 3, 'I-OTHER': 0, 'I-START': 1, 'O': 2}
idx2label = {v: k for k, v in label2idx.items()}
n_labels = len(idx2label)
X_embed = np.load("../embeddings/X_g_embedding_4.npy")


from bert_embedding import BertEmbedding
bert_embedding = BertEmbedding()
X_te = np.asarray(X_embed)

# test_sentence = [['hi hello world joyce','hi joyce']]
X=[]
for seq in paragraphs:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PAD")
    X.append(new_seq)


res = bertSentenceEmbedding(X)

pred_num = model_load.predict(np.asarray(res))
pred = np.argmax(pred_num, axis=-1)
pred_labels = pred2label(pred)

somelist= []
for i, pred in enumerate(pred_labels):
#     print(list(zip(paragraphs[i], pred_labels[i])))
    somelist.append(list(zip(paragraphs[i], pred_labels[i])))

res2 = np.asarray(somelist)  

single_para_list = []
for i in res2:
    for para in i:
        single_para_list.append(para)
        
df_results = pd.DataFrame()
df_results['para'] = [x[0] for x in single_para_list]
df_results['pred_label'] = [x[1] for x in single_para_list]

54


In [15]:
df_results.shape

(281, 2)

In [16]:
df_results.head()

,para,pred_label
0,Chapter 19,O
1,The next day opened a new scene at Longbourn. ...,O
2,"""May I hope, madam, for your interest with you...",B-START
3,Before Elizabeth had time for anything but a b...,O
4,"""Oh dear!—yes—certainly. I am sure Lizzy...",B-OTHER


In [23]:
df_results['para_index'] = data['old_para_index'].values

In [25]:
df_results.to_csv('../../data/predicted/seq_label_ner-v4-pred.csv')

In [20]:
data.shape

(281, 7)